In [2]:
!pip install neurokit2

In [3]:
import pandas as pd
import numpy as np
import wfdb
import ast
import neurokit2 as nk
from wfdb import processing 
from scipy.signal import butter, filtfilt, welch
from scipy.stats import skew, kurtosis

In [4]:
# ------------------------------
# 1. Load Metadata
# ------------------------------
BASE_PATH = "Dataset/ptbxl/"

df_meta = pd.read_csv(BASE_PATH + "ptbxl_database.csv")
df_meta['scp_codes'] = df_meta['scp_codes'].apply(ast.literal_eval)
agg_df = pd.read_csv(BASE_PATH + "scp_statements.csv", index_col=0)
agg_df = agg_df[agg_df['diagnostic'] == 1]
superclasses = ['NORM', 'MI', 'STTC', 'CD', 'HYP']

def get_superclass(scp_dict):
    classes = [agg_df.loc[code]['diagnostic_class'] for code in scp_dict if code in agg_df.index]
    classes = [c for c in classes if c in superclasses]
    return classes[0] if classes else 'OTHER'

df_meta['label'] = df_meta['scp_codes'].apply(get_superclass)

In [5]:

# ------------------------------
# 2. Preprocessing and Filters
# ------------------------------
def bandpass_filter(signal, fs=500, low=0.5, high=40):
    nyq = 0.5 * fs
    b, a = butter(4, [low / nyq, high / nyq], btype='band')
    return filtfilt(b, a, signal)

def preprocess_lead(lead_signal, fs=500):
    filtered = bandpass_filter(lead_signal, fs)
    filtered -= np.median(filtered)        # Baseline correction
    return (filtered - np.mean(filtered)) / np.std(filtered)  # Normalize

# ------------------------------

In [6]:
# 3. Robust Fiducial Detection
# ------------------------------
def detect_rpeaks_wfdb(signal, fs=500):
    """Fallback R-peak detection using WFDB XQRS."""
    try:
        xqrs = processing.XQRS(sig=signal, fs=fs)
        xqrs.detect()
        return np.array(xqrs.qrs_inds)
    except:
        return np.array([])

def robust_delineation(signal, rpeaks, fs=500):
    """Try multiple NeuroKit2 delineation methods."""
    for method in ["dwt", "cwt", "peaks"]:
        try:
            d = nk.ecg_delineate(signal, rpeaks=rpeaks, sampling_rate=fs, method=method)
            if isinstance(d, dict) and any(len(v) > 0 for v in d.values()):
                return d
        except:
            continue
    return {}

def detect_fiducials(signal, fs=500):
    """Hybrid approach: WFDB for R-peaks + NeuroKit2 for delineation."""
    try:
        _, info = nk.ecg_process(signal, sampling_rate=fs)
        rpeaks = info["ECG_R_Peaks"]
    except:
        rpeaks = detect_rpeaks_wfdb(signal, fs)
    d = robust_delineation(signal, rpeaks, fs)
    return rpeaks, d

# ------------------------------

In [7]:
# 4. Helper Functions
# ------------------------------
def safe_get(d, key):
    """Safely extract a list of indices from delineation dict."""
    return [int(x) for x in d.get(key, []) if not np.isnan(x)] if key in d else []

def zero_crossings(signal):
    return ((signal[:-1] * signal[1:]) < 0).sum()

def compute_area(signal, start_idx, end_idx):
    if start_idx is None or end_idx is None or end_idx <= start_idx:
        return np.nan
    return np.trapz(np.abs(signal[start_idx:end_idx]))

def compute_slope(signal, idx1, idx2):
    if idx1 is None or idx2 is None or idx2 <= idx1:
        return np.nan
    return (signal[idx2] - signal[idx1]) / (idx2 - idx1)

def measure_st_elevation(signal, j_point, fs=500):
    if j_point is None: return np.nan
    idx = int(j_point + 0.08 * fs)
    baseline = np.mean(signal[:int(0.2 * fs)])
    return signal[idx] - baseline if idx < len(signal) else np.nan

# ------------------------------

In [8]:
# ------------------------------
# 5. Feature Extraction per Lead
# ------------------------------
def extract_lead_features(lead_signal, fs=500):
    features = {}
    try:
        rpeaks, d = detect_fiducials(lead_signal, fs)
        rr_intervals = np.diff(rpeaks) / fs * 1000

        # Fiducial arrays
        q_on, s_off = safe_get(d, 'ECG_Q_Onsets'), safe_get(d, 'ECG_S_Offsets')
        p_on, t_off = safe_get(d, 'ECG_P_Onsets'), safe_get(d, 'ECG_T_Offsets')
        t_on, t_peaks = safe_get(d, 'ECG_T_Onsets'), safe_get(d, 'ECG_T_Peaks')
        r_pk = safe_get(d, 'ECG_R_Peaks')

        # --- TIME DOMAIN ---
        features['RR_mean'] = np.nanmean(rr_intervals) if len(rr_intervals) else np.nan
        features['RR_std'] = np.nanstd(rr_intervals) if len(rr_intervals) else np.nan
        features['HR'] = 60000 / features['RR_mean'] if features['RR_mean'] else np.nan
        features['QRS_duration'] = np.nanmean([(s - q)/fs*1000 for q, s in zip(q_on, s_off) if s > q]) if q_on and s_off else np.nan
        features['PR_interval'] = np.nanmean([(q - p)/fs*1000 for p, q in zip(p_on, q_on) if q > p]) if p_on and q_on else np.nan
        features['QT_interval'] = np.nanmean([(t - q)/fs*1000 for q, t in zip(q_on, t_off) if t > q]) if q_on and t_off else np.nan
        features['T_duration'] = np.nanmean([(tf - to)/fs*1000 for to, tf in zip(t_on, t_off) if tf > to]) if t_on and t_off else np.nan

        # --- MORPHOLOGICAL ---
        features['R_amp'] = np.nanmax(lead_signal)
        features['S_amp'] = np.nanmin(lead_signal)
        features['R_S_ratio'] = features['R_amp'] / abs(features['S_amp']) if features['S_amp'] != 0 else np.nan
        features['Q_amp'] = np.nanmean([lead_signal[q] for q in q_on if q < len(lead_signal)]) if q_on else np.nan
        features['Q_duration'] = np.nanmean([(r - q)/fs*1000 for q, r in zip(q_on, r_pk) if r > q]) if q_on and r_pk else np.nan
        features['QRS_area'] = np.nanmean([compute_area(lead_signal, q, s) for q, s in zip(q_on, s_off) if s > q]) if q_on and s_off else np.nan

        # T-wave
        if t_peaks:
            t_amps = [lead_signal[t] for t in t_peaks if t < len(lead_signal)]
            features['T_amp'] = np.nanmean(t_amps)
            features['T_polarity'] = np.nanmean([np.sign(a) for a in t_amps])
            features['T_inversion'] = 1 if np.nanmean(t_amps) < 0 else 0
        else:
            features['T_amp'] = features['T_polarity'] = features['T_inversion'] = np.nan

        if t_on and t_off:
            t_asym = []
            for to, tf in zip(t_on, t_off):
                if tf > to:
                    mid = (to + tf) // 2
                    su, sd = compute_slope(lead_signal, to, mid), compute_slope(lead_signal, mid, tf)
                    if sd != 0: t_asym.append(su / abs(sd))
            features['T_asymmetry'] = np.nanmean(t_asym) if t_asym else np.nan

        # ST elevation
        features['ST_elevation'] = np.nanmean([measure_st_elevation(lead_signal, j, fs) for j in s_off]) if s_off else np.nan

        # --- FREQUENCY ---
        f, Pxx = welch(lead_signal, fs, nperseg=1024)
        bands = {'VLF': (0, 0.5), 'LF': (0.5, 4), 'MF': (4, 15), 'HF': (15, 40)}
        for b, (lo, hi) in bands.items():
            features[f'{b}_power'] = np.trapz(Pxx[(f >= lo) & (f <= hi)], f[(f >= lo) & (f <= hi)])
        features['LF_HF_ratio'] = features['LF_power'] / features['HF_power'] if features.get('HF_power', 0) > 0 else np.nan
        features['dominant_freq'] = f[np.argmax(Pxx)]
        Pxx_norm = Pxx / np.sum(Pxx)
        features['spectral_entropy'] = -np.sum(Pxx_norm * np.log2(Pxx_norm + 1e-10))

        # --- STATISTICAL ---
        features['mean'] = np.mean(lead_signal)
        features['median'] = np.median(lead_signal)
        features['std'] = np.std(lead_signal)
        features['skew'] = skew(lead_signal)
        features['kurt'] = kurtosis(lead_signal)
        features['zero_crossings'] = zero_crossings(lead_signal)

    except Exception as e:
        print(f"[WARN] Feature extraction failed: {e}")
    return features



# ------------------------------
# Load ECG Signal Function
# ------------------------------
def load_ecg(record_row):
    """Load ECG signal using WFDB"""
    rec_path = BASE_PATH + record_row['filename_hr']  # uses high-resolution 500Hz files
    sig, fields = wfdb.rdsamp(rec_path)
    return sig, fields['fs'], fields['sig_name']



# ------------------------------

In [9]:
# 6. Process Each ECG Record
# ------------------------------
def extract_record_features(record_row):
    sig, fs, leads = load_ecg(record_row)
    record_features = {}
    for i, lead in enumerate(leads):
        lead_sig = preprocess_lead(sig[:, i])
        lead_feats = extract_lead_features(lead_sig, fs)
        for k, v in lead_feats.items():
            record_features[f'{lead}_{k}'] = v
    record_features['label'] = record_row['label']
    record_features['ecg_id'] = record_row['ecg_id']
    return record_features


# ------------------------------
# 7. Run for 10 Rows
# ------------------------------
all_features = []
for idx, row in df_meta.head(10).iterrows():
    print(f"[INFO] Processing ECG {row['ecg_id']}...")
    feats = extract_record_features(row)
    all_features.append(feats)

df_features = pd.DataFrame(all_features)
df_features.to_csv("ecg_features_test10.csv", index=False)
print("[INFO] Feature extraction (10 records) completed! Saved to ecg_features_test10.csv")

[INFO] Processing ECG 1...
[INFO] Processing ECG 2...
[INFO] Processing ECG 3...
[INFO] Processing ECG 4...
[INFO] Processing ECG 5...
[INFO] Processing ECG 6...
[INFO] Processing ECG 7...
[INFO] Processing ECG 8...
[INFO] Processing ECG 9...
[INFO] Processing ECG 10...
[INFO] Feature extraction (10 records) completed! Saved to ecg_features_test10.csv


In [10]:
import pandas as pd
df= pd.read_csv("ecg_features_test10.csv")
print(df.head(5))

     I_RR_mean   I_RR_std       I_HR  I_QRS_duration  I_PR_interval  \
0   940.000000  16.329932  63.829787             NaN            NaN   
1  1258.285714  76.691803  47.683924             NaN            NaN   
2   940.444444  18.349151  63.799622             NaN            NaN   
3   800.727273  43.386405  74.931880             NaN            NaN   
4   905.200000  48.647302  66.283694             NaN            NaN   

   I_QT_interval  I_T_duration   I_R_amp   I_S_amp  I_R_S_ratio  ...  \
0            NaN           NaN  6.603620 -1.182941     5.582374  ...   
1            NaN           NaN  5.380209 -2.633852     2.042715  ...   
2            NaN           NaN  6.793730 -1.477961     4.596692  ...   
3            NaN           NaN  3.592486 -4.429590     0.811020  ...   
4            NaN           NaN  5.604463 -3.159961     1.773586  ...   

   V6_dominant_freq  V6_spectral_entropy       V6_mean  V6_median  V6_std  \
0          0.976562             4.791125 -2.415845e-17  -0.3139